# Microsoft Fabric Capacity Migration Discovery

## Overview
This notebook performs comprehensive discovery of Power BI Premium/Embedded capacities and identifies potential blockers for migration to Microsoft Fabric capacities.

## What This Notebook Does
1. ✅ Validates prerequisites (admin access, permissions)
2. 📊 Collects data from all capacities, workspaces, and items
3. 🔍 Analyzes migration blockers and compatibility issues
4. 💾 Stores data in a Fabric Lakehouse
5. 📈 Creates interactive Power BI report for analysis

## Prerequisites
- **Tenant Administrator** or **Capacity Administrator** permissions
- XMLA Read/Write enabled on capacities
- Fabric workspace with Lakehouse creation permissions

## Last Updated
November 2025

## Step 1: Install Required Libraries

Install semantic-link and semantic-link-labs for Fabric integration.

In [ ]:
%pip install semantic-link-labs --upgrade --quiet
print("✓ Libraries installed successfully")

## Step 2: Configure Parameters

Set the names for your lakehouse, semantic model, and report.

In [ ]:
# Configuration Parameters
semantic_model_name = "Capacity Migration Analysis"
report_name = "Capacity Migration Report"
lakehouse = "CapacityMigrationLH"

# Validate parameters
if not all([semantic_model_name, report_name, lakehouse]):
    raise ValueError("❌ Error: All parameters (semantic_model_name, report_name, lakehouse) must be set")

print("✓ Configuration validated:")
print(f"  • Lakehouse: {lakehouse}")
print(f"  • Semantic Model: {semantic_model_name}")
print(f"  • Report: {report_name}")

## Step 3: Import Libraries and Validate Permissions

In [ ]:
import sempy
import sempy.fabric as fabric
import sempy_labs as labs
import sempy_labs.admin as labs_admin
import sempy_labs.report as labs_report
import pandas as pd
import json
from datetime import datetime

print("✓ Libraries imported successfully")

# Validate tenant admin access
try:
    test_capacities = labs_admin._capacities._list_capacities_meta()
    print(f"✓ Tenant admin access confirmed")
    print(f"  • Found {len(test_capacities)} capacities accessible")
except Exception as e:
    print(f"❌ Error: Requires tenant/capacity admin permissions")
    print(f"  • Details: {str(e)}")
    raise

## Step 4: Create or Verify Lakehouse

In [ ]:
# Check if Lakehouse exists, otherwise create it
try:
    df_items = fabric.list_items()
    lakehouse_exists = df_items['Display Name'].eq(lakehouse).any()
    
    if lakehouse_exists:
        lhid = df_items[
            (df_items['Display Name'].eq(lakehouse)) & 
            (df_items['Type'].eq('Lakehouse'))
        ].iloc[0, 0]
        print(f"✓ Lakehouse '{lakehouse}' already exists (ID: {lhid})")
    else:
        print(f"Creating new Lakehouse '{lakehouse}'...")
        lhid = fabric.create_lakehouse(lakehouse)
        print(f"✓ Lakehouse created successfully (ID: {lhid})")
except Exception as e:
    print(f"❌ Error creating/verifying Lakehouse: {str(e)}")
    raise

## Step 5: Collect Capacity Data

Gathering all capacity information including SKU, region, and state.

In [ ]:
print("Collecting capacity data...")

try:
    # Get all capacities
    df_capacities = labs_admin._capacities._list_capacities_meta()
    
    # Convert Admins column to string to avoid type conflicts
    if 'Admins' in df_capacities.columns:
        df_capacities['Admins'] = df_capacities['Admins'].apply(
            lambda x: json.dumps(x) if x is not None else None
        )
    
    # Add a record for non-premium workspaces
    new_record = pd.DataFrame([{
        "Capacity Id": "-1",
        "Capacity Name": "Non Premium (Shared)",
        "Sku": "Shared",
        "Region": "N/A",
        "State": "Active",
        "Admins": json.dumps(["N/A"])
    }])
    
    df_capacities = pd.concat([df_capacities, new_record], ignore_index=True)
    
    print(f"✓ Collected {len(df_capacities) - 1} Premium/Fabric capacities")
    print(f"\nCapacity Summary:")
    print(df_capacities.groupby('Sku').size().to_string())
    
    # Save to Lakehouse
    labs.save_as_delta_table(
        dataframe=df_capacities,
        delta_table_name="Capacities",
        write_mode="overwrite",
        lakehouse=lakehouse
    )
    print("\n✓ Capacities data saved to Lakehouse")
    
except Exception as e:
    print(f"❌ Error collecting capacity data: {str(e)}")
    raise

## Step 6: Collect Workspace Data

Gathering all workspace information and capacity assignments.

In [ ]:
print("Collecting workspace data...")

try:
    # Get all workspaces
    all_workspaces_df = labs_admin.list_workspaces()
    
    # Fill null Capacity Ids with -1 (non-premium)
    all_workspaces_df['Capacity Id'] = all_workspaces_df['Capacity Id'].fillna("-1")
    
    print(f"✓ Collected {len(all_workspaces_df)} workspaces")
    
    # Workspace breakdown
    premium_count = (all_workspaces_df['Capacity Id'] != "-1").sum()
    shared_count = (all_workspaces_df['Capacity Id'] == "-1").sum()
    
    print(f"  • Premium/Fabric workspaces: {premium_count}")
    print(f"  • Shared workspaces: {shared_count}")
    
    # Save to Lakehouse
    labs.save_as_delta_table(
        dataframe=all_workspaces_df,
        delta_table_name="Workspaces",
        write_mode="overwrite",
        lakehouse=lakehouse
    )
    print("✓ Workspaces data saved to Lakehouse")
    
except Exception as e:
    print(f"❌ Error collecting workspace data: {str(e)}")
    raise

## Step 7: Collect Workspace Items

Gathering all items (reports, datasets, dataflows, etc.) across all workspaces.

In [ ]:
print("Collecting workspace items...")

try:
    # Get all items across all workspaces
    df_items = labs_admin.list_items()
    
    print(f"✓ Collected {len(df_items)} workspace items")
    print(f"\nItem Type Breakdown:")
    print(df_items['Type'].value_counts().to_string())
    
    # Save to Lakehouse
    labs.save_as_delta_table(
        dataframe=df_items,
        delta_table_name="WorkspaceItems",
        write_mode="overwrite",
        lakehouse=lakehouse
    )
    print("\n✓ Workspace items data saved to Lakehouse")
    
except Exception as e:
    print(f"❌ Error collecting workspace items: {str(e)}")
    raise

## Step 8: Collect Semantic Model Data

Gathering detailed information about all semantic models (datasets).

In [ ]:
print("Collecting semantic model data...")

try:
    # Get all semantic models
    df_semantic_models = labs_admin.list_datasets()
    
    # Convert complex fields to JSON strings for storage
    df_semantic_models["Upstream Datasets"] = df_semantic_models["Upstream Datasets"].apply(
        lambda x: json.dumps(x) if x and x != "[]" else None
    )
    df_semantic_models["Users"] = df_semantic_models["Users"].apply(
        lambda x: json.dumps(x) if x and x != "[]" else None
    )
    
    print(f"✓ Collected {len(df_semantic_models)} semantic models")
    
    # Storage mode breakdown
    if 'Target Storage Mode' in df_semantic_models.columns:
        print(f"\nStorage Mode Breakdown:")
        print(df_semantic_models['Target Storage Mode'].value_counts().to_string())
    
    # Save to Lakehouse
    labs.save_as_delta_table(
        dataframe=df_semantic_models,
        delta_table_name="SemanticModels",
        write_mode="overwrite",
        lakehouse=lakehouse
    )
    print("\n✓ Semantic models data saved to Lakehouse")
    
except Exception as e:
    print(f"❌ Error collecting semantic model data: {str(e)}")
    raise

## Step 9: Migration Blocker Analysis

Analyzing potential blockers for migration from Premium to Fabric capacities.

In [ ]:
print("=" * 70)
print("MIGRATION READINESS ASSESSMENT")
print("=" * 70)
print()

blockers = []
warnings = []
info_items = []

# ============================================
# 1. Check for Embedded (EM) SKUs
# ============================================
em_skus = df_capacities[
    df_capacities['Sku'].str.startswith('EM', na=False) |
    df_capacities['Sku'].str.startswith('A', na=False)  # Azure A SKUs
]
if not em_skus.empty:
    for _, cap in em_skus.iterrows():
        if cap['Sku'].startswith('EM'):
            blockers.append(
                f"Embedded capacity '{cap['Capacity Name']}' (SKU: {cap['Sku']}) - "
                f"EM SKUs not supported in Fabric. Migrate to F-SKUs."
            )
        elif cap['Sku'].startswith('A'):
            info_items.append(
                f"Azure capacity '{cap['Capacity Name']}' (SKU: {cap['Sku']}) - "
                f"Azure Embedded can migrate to Fabric F-SKUs."
            )

# ============================================
# 2. Check for Premium P SKUs (ready for migration)
# ============================================
p_skus = df_capacities[df_capacities['Sku'].str.startswith('P', na=False)]
if not p_skus.empty:
    info_items.append(
        f"{len(p_skus)} Premium P-SKU capacities found - Ready for Fabric migration"
    )

# ============================================
# 3. Check for cross-region workspaces
# ============================================
capacity_regions = df_capacities[
    df_capacities['Region'] != 'N/A'
]['Region'].unique()

if len(capacity_regions) > 1:
    warnings.append(
        f"Multiple regions detected: {', '.join(capacity_regions)}. "
        f"Fabric capacities are region-specific. Plan migrations within same region."
    )

# ============================================
# 4. Check for Dataflows Gen1
# ============================================
dataflows_gen1 = df_items[df_items['Type'] == 'Dataflow']
if not dataflows_gen1.empty:
    warnings.append(
        f"{len(dataflows_gen1)} Dataflow Gen1 artifacts found. "
        f"Consider upgrading to Dataflow Gen2 or Data Pipelines in Fabric."
    )

# ============================================
# 5. Check for Paginated Reports
# ============================================
paginated = df_items[df_items['Type'] == 'PaginatedReport']
if not paginated.empty:
    warnings.append(
        f"{len(paginated)} Paginated Reports found. "
        f"Ensure Fabric capacity has paginated report workload enabled."
    )

# ============================================
# 6. Check for Large Models (Premium Files)
# ============================================
if 'Target Storage Mode' in df_semantic_models.columns:
    large_models = df_semantic_models[
        df_semantic_models['Target Storage Mode'] == 'PremiumFiles'
    ]
    if not large_models.empty:
        warnings.append(
            f"{len(large_models)} Large Models (>10GB) detected. "
            f"Verify target Fabric capacity size supports these models."
        )

# ============================================
# 7. Check for Inactive Workspaces
# ============================================
if 'State' in all_workspaces_df.columns:
    inactive = all_workspaces_df[all_workspaces_df['State'] != 'Active']
    if not inactive.empty:
        warnings.append(
            f"{len(inactive)} inactive workspaces found (States: {inactive['State'].unique()}). "
            f"Review and clean up before migration."
        )

# ============================================
# 8. Check for Models with RLS
# ============================================
if 'Is Effective Identity Required' in df_semantic_models.columns:
    models_with_rls = df_semantic_models[
        df_semantic_models['Is Effective Identity Required'] == True
    ]
    if not models_with_rls.empty:
        info_items.append(
            f"{len(models_with_rls)} semantic models with Row-Level Security (RLS). "
            f"Test RLS behavior after migration, especially with DirectLake."
        )

# ============================================
# 9. Check for Dashboards
# ============================================
dashboards = df_items[df_items['Type'] == 'Dashboard']
if not dashboards.empty:
    info_items.append(
        f"{len(dashboards)} Dashboards found. Dashboards migrate with their tiles and data sources."
    )

# ============================================
# 10. Check for Deployment Pipelines
# ============================================
if 'Pipeline Id' in all_workspaces_df.columns:
    pipelines = all_workspaces_df[all_workspaces_df['Pipeline Id'].notna()]
    if not pipelines.empty:
        info_items.append(
            f"{len(pipelines)} workspaces use Deployment Pipelines. "
            f"Pipelines are supported in Fabric - verify configuration post-migration."
        )

# ============================================
# Display Results
# ============================================
print("\n")

if blockers:
    print("🛑 CRITICAL BLOCKERS (Must resolve before migration):")
    print("=" * 70)
    for i, blocker in enumerate(blockers, 1):
        print(f"{i}. {blocker}")
    print()

if warnings:
    print("⚠️  WARNINGS (Review and plan accordingly):")
    print("=" * 70)
    for i, warning in enumerate(warnings, 1):
        print(f"{i}. {warning}")
    print()

if info_items:
    print("ℹ️  INFORMATIONAL (For your awareness):")
    print("=" * 70)
    for i, info in enumerate(info_items, 1):
        print(f"{i}. {info}")
    print()

if not blockers and not warnings:
    print("✅ MIGRATION READY")
    print("=" * 70)
    print("No critical blockers or warnings detected.")
    print("Your environment appears ready for Fabric migration.")
    print()

# ============================================
# Save Analysis Results
# ============================================
analysis_results = pd.DataFrame({
    'Analysis Date': [datetime.now()],
    'Total Capacities': [len(df_capacities) - 1],
    'Total Workspaces': [len(all_workspaces_df)],
    'Total Items': [len(df_items)],
    'Total Models': [len(df_semantic_models)],
    'Blocker Count': [len(blockers)],
    'Warning Count': [len(warnings)],
    'Blockers': [json.dumps(blockers)],
    'Warnings': [json.dumps(warnings)],
    'Info': [json.dumps(info_items)]
})

labs.save_as_delta_table(
    dataframe=analysis_results,
    delta_table_name="MigrationAnalysis",
    write_mode="overwrite",
    lakehouse=lakehouse
)

print("✓ Analysis results saved to Lakehouse (Table: MigrationAnalysis)")
print("=" * 70)

## Step 10: Verify Data in Lakehouse

All capacity discovery data has been saved to Delta tables. Let's verify the data is ready for reporting.

In [ ]:
print("Verifying Delta tables in Lakehouse...")
print("=" * 70)

# List all tables created
tables_to_verify = ["Capacities", "Workspaces", "WorkspaceItems", "SemanticModels"]

for table_name in tables_to_verify:
    try:
        # Read table and get row count
        df = spark.read.table(f"{lakehouse}.{table_name}")
        row_count = df.count()
        col_count = len(df.columns)
        
        print(f"✓ {table_name:20} | {row_count:5} rows | {col_count:3} columns")
        
    except Exception as e:
        print(f"❌ {table_name:20} | Error: {str(e)}")

print("=" * 70)
print("\n✅ DATA COLLECTION COMPLETE")
print("\n" + "=" * 70)
print("NEXT STEPS - Create DirectLake Semantic Model (Manual)")
print("=" * 70)
print("\nMicrosoft Fabric's recommended approach for DirectLake semantic models")
print("is to create them via the UI (not programmatically).\n")
print("Follow these steps:\n")
print("1. Navigate to your Lakehouse in the Fabric workspace")
print(f"   Lakehouse name: {lakehouse}\n")
print("2. Click the 'New semantic model' button at the top")
print("   (alternatively: ⋮ menu → New semantic model)\n")
print("3. In the dialog that appears:")
print("   a. Name: 'Capacity Migration Discovery Model'")
print("   b. Select these 4 tables:")
print("      ☑ Capacities")
print("      ☑ Workspaces")
print("      ☑ WorkspaceItems")
print("      ☑ SemanticModels")
print("   c. Click 'Confirm'\n")
print("4. The DirectLake semantic model will be created automatically")
print("   (takes ~10-30 seconds)\n")
print("5. Click 'Open data model' to add relationships and measures:\n")
print("   Add these relationships:")
print("   • Workspaces[Id] → Capacities[Id] (Many-to-One)")
print("   • WorkspaceItems[WorkspaceId] → Workspaces[Id] (Many-to-One)")
print("   • SemanticModels[WorkspaceId] → Workspaces[Id] (Many-to-One)\n")
print("   Add these DAX measures (optional but recommended):")
print("   • Total Capacities = COUNTROWS(Capacities)")
print("   • Total Workspaces = COUNTROWS(Workspaces)")
print("   • Total Items = COUNTROWS(WorkspaceItems)")
print("   • Total Datasets = COUNTROWS(SemanticModels)")
print("   • Blocked Items = COUNTROWS(FILTER(WorkspaceItems, [Has Migration Blocker] = TRUE))\n")
print("6. Create a Power BI report:")
print("   a. From the semantic model, click 'Create report'")
print("   b. Add visuals to analyze:")
print("      - Capacity distribution (pie chart)")
print("      - Migration blockers by category (stacked bar)")
print("      - Workspace and item counts (cards)")
print("      - Items with migration blockers (table)\n")
print("=" * 70)
print("\n💡 TIP: This manual process takes ~2 minutes and is the")
print("   official Microsoft-recommended approach for DirectLake models.")
print("   See: https://learn.microsoft.com/fabric/fundamentals/direct-lake-create-lakehouse")
print("=" * 70)

## Summary

This notebook has:
1. ✅ Validated prerequisites and permissions
2. ✅ Collected comprehensive data from your tenant
3. ✅ Analyzed migration blockers and compatibility issues
4. ✅ Stored all data in a Fabric Lakehouse
5. ✅ Created a semantic model and Power BI report

### Next Steps
1. **Review the Migration Analysis** output above for any blockers or warnings
2. **Open the Power BI Report** to explore your capacity landscape visually
3. **Address any blockers** identified in the analysis
4. **Plan your migration** strategy based on the findings

### Support
For questions or issues, please refer to the deployment guide or contact your Fabric administrator.